### Loading the Data from Lakehouse

In [1]:
from pyspark.sql.functions import *

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 3, Finished, Available, Finished)

In [2]:
df= spark.sql("select  * from bing_data_lakehouse.today_news_in_india")

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 4, Finished, Available, Finished)

In [4]:
display(df)

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8fa0da44-2f42-44ce-8bfd-f4ab1941978a)

### Implementing Sentiment Analysis using Synapse ML

In [5]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 7, Finished, Available, Finished)

#### Model training

In [6]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 8, Finished, Available, Finished)

In [7]:
result = model.transform(df)

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 9, Finished, Available, Finished)

In [8]:
display(result)

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a475c917-54be-45d6-9515-434ad673f19d)

In [9]:
result =result.withColumn('Sentiment',col('response.documents.sentiment')).drop(col('response'))

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 11, Finished, Available, Finished)

In [10]:
result= result.drop('error')

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 12, Finished, Available, Finished)

In [11]:
display(result)

StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5b0c83cd-c0e2-45d6-9316-f2c978896412)

### Saving the Data incrementally

In [13]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_data_lakehouse.news_sentiment_analysis'

    result.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    result.createOrReplaceTempView("vw_sentiment_result")

    spark.sql(f""" Merge into {table_name} t
                using vw_sentiment_result s
                on t.url= s.url
                when matched and 
                    s.title <> t.title or
                    s.description <> t.description OR
                    s.category <> t.category
                then update set *
                when not matched then
                INSERT *""")



StatementMeta(, aab3fe37-a989-4ea0-8999-83a42d4413e8, 15, Finished, Available, Finished)

Table Already Exists
